# Transfer Learning

In this notebook, you'll learn how to use pre-trained networks to solved challenging problems in computer vision. Specifically, you'll use networks trained on [ImageNet](http://www.image-net.org/) [available from torchvision](http://pytorch.org/docs/0.3.0/torchvision/models.html). 

ImageNet is a massive dataset with over 1 million labeled images in 1000 categories. It's used to train deep neural networks using an architecture called convolutional layers. I'm not going to get into the details of convolutional networks here, but if you want to learn more about them, please [watch this](https://www.youtube.com/watch?v=2-Ol7ZB0MmU).

Once trained, these models work astonishingly well as feature detectors for images they weren't trained on. Using a pre-trained network on images not in the training set is called transfer learning. Here we'll use transfer learning to train a network that can classify our cat and dog photos with near perfect accuracy.

With `torchvision.models` you can download these pre-trained networks and use them in your applications. We'll include `models` in our imports now.

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

In [2]:
torch.cuda.is_available()

True

Most of the pretrained models require the input to be 224x224 images. Also, we'll need to match the normalization used when the models were trained. Each color channel was normalized separately, the means are `[0.485, 0.456, 0.406]` and the standard deviations are `[0.229, 0.224, 0.225]`.

In [3]:
data_dir = 'data'

# TODO: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor()])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

We can load in a model such as [DenseNet](http://pytorch.org/docs/0.3.0/torchvision/models.html#id5). Let's print out the model architecture so we can see what's going on.

In [4]:
model = models.densenet121(pretrained=True)
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

This model is built out of two main parts, the features and the classifier. The features part is a stack of convolutional layers and overall works as a feature detector that can be fed into a classifier. The classifier part is a single fully-connected layer `(classifier): Linear(in_features=1024, out_features=1000)`. This layer was trained on the ImageNet dataset, so it won't work for our specific problem. That means we need to replace the classifier, but the features will work perfectly on their own. In general, I think about pre-trained networks as amazingly good feature detectors that can be used as the input for simple feed-forward classifiers.

In [5]:
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier

With our model built, we need to train the classifier. However, now we're using a **really deep** neural network. If you try to train this on a CPU like normal, it will take a long, long time. Instead, we're going to use the GPU to do the calculations. The linear algebra computations are done in parallel on the GPU leading to 100x increased training speeds. It's also possible to train on multiple GPUs, further decreasing training time.

PyTorch, along with pretty much every other deep learning framework, uses [CUDA](https://developer.nvidia.com/cuda-zone) to efficiently compute the forward and backwards passes on the GPU. In PyTorch, you move your model parameters and other tensors to the GPU memory using `model.to('cuda')`. You can move them back from the GPU with `model.to('cpu')` which you'll commonly do when you need to operate on the network output outside of PyTorch. As a demonstration of the increased speed, I'll compare how long it takes to perform a forward and backward pass with and without a GPU.

In [6]:
import time

In [7]:
for device in ['cpu', 'cuda']:

    criterion = nn.NLLLoss()
    # Only train the classifier parameters, feature parameters are frozen
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

    model.to(device)

    for ii, (inputs, labels) in enumerate(trainloader):

        # Move input and label tensors to the GPU
        inputs, labels = inputs.to(device), labels.to(device)

        start = time.time()

        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if ii==3:
            break
        
    print(f"Device = {device}; Time per batch: {(time.time() - start)/3:.3f} seconds")

Device = cpu; Time per batch: 3.764 seconds
Device = cuda; Time per batch: 0.008 seconds


You can write device agnostic code which will automatically use CUDA if it's enabled like so:
```python
# at beginning of the script
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

...

# then whenever you get a new Tensor or Module
# this won't copy if they are already on the desired device
input = data.to(device)
model = MyModule(...).to(device)
```

From here, I'll let you finish training the model. The process is the same as before except now your model is much more powerful. You should get better than 95% accuracy easily.

>**Exercise:** Train a pretrained models to classify the cat and dog images. Continue with the DenseNet model, or try ResNet, it's also a good model to try out first. Make sure you are only training the classifier and the parameters for the features part are frozen.

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [27]:
model = models.resnet50(pretrained=True)

In [28]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [29]:
for param in model.parameters():
    param.requires_grad = False

In [30]:
from collections import OrderedDict

In [31]:
cls = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(2048, 512)),
    ('relu', nn.ReLU()),
    ('fc2', nn.Linear(512, 2)),
    ('output', nn.LogSoftmax(dim=1))
]))

In [32]:
model.fc = cls

In [33]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.003)

In [36]:
def train(model, trainloader, testloader, criterion, optimizer, device, epochs=5):
    model.to(device)
    for e in range(epochs):
        print("Starting training on epoch {}".format(e))
        running_loss = 0
        model.train()
        for k, train in enumerate(trainloader):
            images, labels = train
            print("Train: Running batch {}/{}".format(k, len(trainloader)))
            
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            logps = model(images)
            loss = criterion(logps, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        else:
            with torch.no_grad():
                print("Starting evaluation")
                model.eval()
                test_loss = 0
                acc = 0
                for k, test in enumerate(testloader):
                    images, labels = test
                    print("Test: Running batch {}/{}".format(k, len(testloader)))
                    images, labels = images.to(device), labels.to(device)

                    logps = model(images)
                    test_loss += criterion(logps, labels)

                    prob = torch.exp(logps)

                    top_p, top_class = prob.topk(1, dim=1)

                    equals = top_class == labels.view(*top_class.shape)
                    acc += torch.mean(equals.type(torch.FloatTensor))
            
            print("---------------------------------")
            print("Epoch: {}/{}".format(e+1, epochs))
            print("Train loss: {}".format(running_loss/len(trainloader)))
            print("Test loss: {}".format(test_loss/len(testloader)))
            print("Acc: {}".format(acc/len(testloader)))

In [39]:
train(model, trainloader, testloader, criterion, optimizer, device, epochs=1)

Starting training on epoch 0
Train: Running batch 0/352
Train: Running batch 1/352
Train: Running batch 2/352
Train: Running batch 3/352
Train: Running batch 4/352
Train: Running batch 5/352
Train: Running batch 6/352
Train: Running batch 7/352
Train: Running batch 8/352
Train: Running batch 9/352
Train: Running batch 10/352
Train: Running batch 11/352
Train: Running batch 12/352
Train: Running batch 13/352
Train: Running batch 14/352
Train: Running batch 15/352
Train: Running batch 16/352
Train: Running batch 17/352
Train: Running batch 18/352
Train: Running batch 19/352
Train: Running batch 20/352
Train: Running batch 21/352
Train: Running batch 22/352
Train: Running batch 23/352
Train: Running batch 24/352
Train: Running batch 25/352
Train: Running batch 26/352
Train: Running batch 27/352
Train: Running batch 28/352
Train: Running batch 29/352
Train: Running batch 30/352
Train: Running batch 31/352
Train: Running batch 32/352
Train: Running batch 33/352
Train: Running batch 34/352
T

Train: Running batch 286/352
Train: Running batch 287/352
Train: Running batch 288/352
Train: Running batch 289/352
Train: Running batch 290/352
Train: Running batch 291/352
Train: Running batch 292/352
Train: Running batch 293/352
Train: Running batch 294/352
Train: Running batch 295/352
Train: Running batch 296/352
Train: Running batch 297/352
Train: Running batch 298/352
Train: Running batch 299/352
Train: Running batch 300/352
Train: Running batch 301/352
Train: Running batch 302/352
Train: Running batch 303/352
Train: Running batch 304/352
Train: Running batch 305/352
Train: Running batch 306/352
Train: Running batch 307/352
Train: Running batch 308/352
Train: Running batch 309/352
Train: Running batch 310/352
Train: Running batch 311/352
Train: Running batch 312/352
Train: Running batch 313/352
Train: Running batch 314/352
Train: Running batch 315/352
Train: Running batch 316/352
Train: Running batch 317/352
Train: Running batch 318/352
Train: Running batch 319/352
Train: Running